Build a ResNet based Convolutional Neural Network, like what we built in lectures (with skip connections), to classify the images across all 10 classes in CIFAR 10. For this problem, let’s use 10 blocks for ResNet and call it ResNet-10. Use the similar dimensions and channels as we need in lectures. Train your network for 200 epochs.

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time

datapath = '../data-unversions/p1ch7/'
cifar10 = datasets.CIFAR10(root= datapath, train=True, download = True, transform=transforms.ToTensor())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 170M/170M [00:23<00:00, 7.18MB/s]


Extracting ../data-unversions/p1ch7/cifar-10-python.tar.gz to ../data-unversions/p1ch7/


Pre-processing. Seperating training and validation datasets.

In [2]:
imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)])

train_data = datasets.CIFAR10(root= datapath, train=True, download = True, transform=normalize)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

valid_data = datasets.CIFAR10(root= datapath, train=False, download = True, transform=normalize)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


Batch Normalization ResNet Training and Validation

In [ ]:
class Block(nn.Module):
    def __init__(self, inputs, outputs, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(inputs, outputs, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(outputs)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(outputs, outputs, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(outputs)
        self.downsample = nn.Sequential()
        if stride != 1 or inputs != outputs:
            self.downsample = nn.Sequential(
                nn.Conv2d(inputs, outputs, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outputs),
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class ResNet10(nn.Module):
    def __init__(self, block, layers):
        super(ResNet10, self).__init__()
        self.inputs = 16
        self.conv1 = nn.Conv2d(3, self.inputs, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inputs)
        self.relu = nn.ReLU(inplace=True)
        self.blocks = self._make_layers(block, layers)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(self.inputs, 10)

    def _make_layers(self, block, layers):
        layers_list = []
        for layer in layers:
            layers_list.append(block(self.inputs, self.inputs, stride=1))
        return nn.Sequential(*layers_list)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.blocks(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Initialize the model with Batch Normalization
model = ResNet10(Block, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
epochs = 200
total_start_time = time.time()
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    losses = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {losses/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time

# Validation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

valid_accuracy = correct / total

print(f'Batch Normalization Training Time: {total_training_time:.2f} seconds')
print(f'Batch Normalization Final Training Loss: {losses / len(train_loader)}')
print(f'Batch Normalization Final Validation Accuracy: {valid_accuracy * 100:.2f}%')

Epoch 1/200, Loss: 1.677608071385747, Training Time: 19.85 seconds
Epoch 11/200, Loss: 0.8672370893113753, Training Time: 18.88 seconds
Epoch 21/200, Loss: 0.7534167638901249, Training Time: 18.55 seconds
Epoch 31/200, Loss: 0.6868878088872451, Training Time: 18.62 seconds
Epoch 41/200, Loss: 0.647835611306188, Training Time: 18.30 seconds
Epoch 51/200, Loss: 0.6142463703899432, Training Time: 18.03 seconds
Epoch 61/200, Loss: 0.589586685342557, Training Time: 17.88 seconds
Epoch 71/200, Loss: 0.5637577331584432, Training Time: 18.04 seconds
Epoch 81/200, Loss: 0.5478988943426201, Training Time: 18.42 seconds
Epoch 91/200, Loss: 0.5297096873945593, Training Time: 18.35 seconds
Epoch 101/200, Loss: 0.5165627595523129, Training Time: 18.52 seconds
Epoch 111/200, Loss: 0.5006466954374862, Training Time: 18.26 seconds
Epoch 121/200, Loss: 0.48845816460792973, Training Time: 17.83 seconds
Epoch 131/200, Loss: 0.4741320013999939, Training Time: 17.76 seconds
Epoch 141/200, Loss: 0.4715070529